In [71]:
!pip install sentence-transformers faiss-cpu PyPDF2 requests


In [72]:
import os
import numpy as np
import faiss
import requests
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer


In [73]:
print("Loading embedding model...")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
print("Embedding model loaded.")


Loading embedding model...


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Embedding model loaded.


In [74]:
def extract_text_with_metadata(pdf_path):
    reader = PdfReader(pdf_path)
    pages = []
    
    for page_number, page in enumerate(reader.pages):
        text = page.extract_text()
        if text:
            pages.append({
                "page_number": page_number,
                "text": text
            })
    return pages


In [75]:
def chunk_pages(pages, pdf_name, chunk_size=300, overlap=50):
    metadata_store = []
    
    for page in pages:
        text = page["text"]
        page_number = page["page_number"]
        
        start = 0
        chunk_index = 0
        
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]
            
            metadata_store.append({
                "chunk_id": f"{pdf_name}_page_{page_number}_chunk_{chunk_index}",
                "document_name": pdf_name,
                "page_number": page_number,
                "chunk_index": chunk_index,
                "text": chunk_text
            })
            
            start += chunk_size - overlap
            chunk_index += 1
            
    return metadata_store


In [76]:
pdf_path = "sample.pdf"
pdf_name = os.path.basename(pdf_path)

pages = extract_text_with_metadata(pdf_path)
metadata_store = chunk_pages(pages, pdf_name)

print("Total chunks:", len(metadata_store))


Total chunks: 6


In [77]:
chunk_texts = [item["text"] for item in metadata_store]

embeddings = embedding_model.encode(chunk_texts)
embeddings = np.array(embeddings).astype("float32")

faiss.normalize_L2(embeddings)

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

print("FAISS index built successfully.")


FAISS index built successfully.


In [78]:
def retrieve(query, k=3):
    query_embedding = embedding_model.encode([query])
    query_embedding = np.array(query_embedding).astype("float32")
    faiss.normalize_L2(query_embedding)
    
    distances, indices = index.search(query_embedding, k)
    
    results = []
    for idx in indices[0]:
        results.append(metadata_store[idx])
    
    return results


In [79]:
def build_prompt(query, retrieved_chunks):
    context = "\n\n".join(
        [chunk["text"] for chunk in retrieved_chunks]
    )
    
    prompt = f"""
You are a helpful assistant.

Use ONLY the provided context to answer the question.
If the answer is not in the context, say "I don't know."

Context:
{context}

Question:
{query}

Answer:
"""
    return prompt


In [80]:
def generate_answer(prompt):
    url = "http://localhost:11434/api/generate"
    
    payload = {
        "model": "mistral",
        "prompt": prompt,
        "stream": False
    }
    
    response = requests.post(url, json=payload)
    
    return response.json()["response"]


In [81]:
def rag_pipeline(query):
    print("\nUser Question:", query)
    
    retrieved_chunks = retrieve(query, k=3)
    
    print("\nRetrieved Context Preview:\n")
    for i, chunk in enumerate(retrieved_chunks):
        print(f"Chunk {i+1} (Page {chunk['page_number']}):")
        print(chunk["text"][:200])
        print("-" * 50)
    
    prompt = build_prompt(query, retrieved_chunks)
    answer = generate_answer(prompt)
    
    print("\nFinal Answer:\n")
    print(answer)
    
    return answer


In [82]:
rag_pipeline("What are applications of machine learning?")



User Question: What are applications of machine learning?

Retrieved Context Preview:

Chunk 1 (Page 0):
eled data, and reinforcement learning
learns through rewards and penalties. Applications of machine learning include recommendation
systems, fraud detection, natural language processing, computer visi
--------------------------------------------------
Chunk 2 (Page 0):
Introduction to Machine Learning
Machine learning is a subset of artificial intelligence that focuses on building systems that learn from
data. Instead of being explicitly programmed with rules, machi
--------------------------------------------------
Chunk 3 (Page 0):
sions based on historical information. There are three main types of machine learning:
supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses
labeled data, un
--------------------------------------------------

Final Answer:

 Applications of machine learning include recommendation systems, fraud detection, nat

' Applications of machine learning include recommendation systems, fraud detection, natural language processing, computer vision, and predictive analytics.'